In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

## Data Load

In [5]:
start = datetime.now()
if not os.path.isfile('metadata.csv'):
    data = open('metadata.csv', mode='w')
    row = list()
    files=['D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_1.txt','D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_2.txt', 
           'D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_3.txt', 'D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_4.txt']
    for file in files:
        print("Reading ratings from {}...".format(file))
        with open(file) as f:
            for line in f: 
                del row[:] # you don't have to do this.
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        print("Done.\n")
    data.close()
print('Time taken :', datetime.now() - start)

Reading ratings from D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_1.txt...
Done.

Reading ratings from D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_2.txt...
Done.

Reading ratings from D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_3.txt...
Done.

Reading ratings from D:/12S17037_Nita S W Sirait_Sistem Informasi/Kuliah/semester 7/dami/proyek/combined_data_4.txt...
Done.

Time taken : 0:07:39.645830


In [6]:
print("membuat dataframe metadata.csv")
df = pd.read_csv('metadata.csv', sep=',', 
                       names=['movie', 'user','rating','date'])
df.date = pd.to_datetime(df.date)
print('Done..\n')

# we are arranging the ratings according to time.
print('Mengurutkan dataframe berdasarkan tanggal')
df.sort_values(by='date', inplace=True)
print('Done..')

membuat dataframe metadata.csv
Done..

Mengurutkan dataframe berdasarkan tanggal
Done..


In [7]:
df.head()

,movie,user,rating,date
56431994,10341,510180,4,1999-11-11
9056171,1798,510180,5,1999-11-11
58698779,10774,510180,3,1999-11-11
48101611,8651,510180,2,1999-11-11
81893208,14660,510180,2,1999-11-11


In [8]:
df.tail()

,movie,user,rating,date
49939086,8993,2183787,4,2005-12-31
42072268,7430,258170,4,2005-12-31
47098649,8467,1534359,5,2005-12-31
55621336,10168,2543295,2,2005-12-31
25464092,4736,1346243,5,2005-12-31


## Cek NaN

In [9]:
print("tidak ada NaN pada dataframe : ", sum(df.isnull().any()))

tidak ada NaN pada dataframe :  0


## Cek Jumlah Baris Data 

In [13]:
print("Total data ")
print("\nTotal ratings :",df.shape[0])
print("Total Users   :", len(np.unique(df.user)))
print("Total movies  :", len(np.unique(df.movie)))

Total data 

Total ratings : 100480507
Total Users   : 480189
Total movies  : 17770


## Menghapus Nilai yang duplikat

In [14]:
dup_bool = df.duplicated(['movie','user','rating'])
dups = sum(dup_bool) # by considering all columns..( including timestamp)
print("Terdapat {} Nilai rating pada metadata..".format(dups))

Terdapat 0 Nilai rating pada metadata..
